In [1]:
from datetime import datetime
from calendar import monthrange, month_name
from pathlib import Path

import os
import numpy as np
import pandas as pd
import netCDF4 as nc
import xarray as xr

import time
import pickle
import cdsapi
import math

# Interpolation
from scipy.interpolate import RegularGridInterpolator, LinearNDInterpolator
import scipy.interpolate.interpnd

from tqdm import tqdm
tqdm.pandas()

In [2]:
# Set data frame options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
load_cygnss_boolean = False
load_smap_boolean = True
interpolate_dfs_boolean = True

In [ ]:
### CREATE CYGNSS DATAFRAME ###

In [4]:
def calculate_sr_value(snr, p_r, g_t, g_r, d_ts, d_sr):
    # snr(dB), p_r(dBW), g_t(dBi), g_r(dBi), d_ts(meter), d_sr(meter)
    return snr - p_r - g_t - g_r - (20 * np.log10(0.19)) + (20 * np.log10(d_ts + d_sr)) + (20 * np.log10(4 * np.pi))


def compute_surface_reflectivity(df):
    df['sr'] = df.apply(
        lambda row: calculate_sr_value(row.ddm_snr, row.gps_tx_power_db_w, row.gps_ant_gain_db_i, row.sp_rx_gain,
                                       row.tx_to_sp_range, row.rx_to_sp_range), axis=1)
    return df


def calculate_hours_after_jan_value(day_of_year, ddm_timestamp):
    return (day_of_year - 1) * 24 + round(ddm_timestamp / (60 * 60))


def compute_hours_after_jan(df):
    df['hours_after_jan_2020'] = df.apply(
        lambda row: calculate_hours_after_jan_value(row.day_of_year, row.ddm_timestamp_utc), axis=1)
    return df


def generate_unique_track_id_value(track_id, day_of_year, prn_nr, sat_nr):
    return track_id * 10000 + prn_nr * 10 + sat_nr + day_of_year/1000


def compute_unique_track_ids(df):
    df['unique_track_id'] = df.apply(
        lambda row: generate_unique_track_id_value(row.track_id, row.day_of_year, row.prn_code, row.spacecraft_num), axis=1)
    return df


def generate_qf_list(qf_number):
    qf_list = []
    binary = format(qf_number, 'b')
    for i in range(len(binary)):
        if binary[i] == '1':
            qf_list.append(2 ** (int(i)))

    return qf_list


def compute_prn_to_block_value(prn_code):
    iir_list = [2, 13, 16, 19, 20, 21, 22]
    iif_list = [1, 3, 6, 8, 9, 10, 25, 26, 27, 30, 32]
    iir_m_list = [5, 7, 12, 15, 17, 29, 31]
    iii_list = [4, 11, 14, 18, 23, 24]
    
    if prn_code in iir_list:
        return 'IIR'
    elif prn_code in iif_list:
        return 'IIF'
    elif prn_code in iir_m_list:
        return 'IIR-M'
    elif prn_code in iii_list:
        return 'III'
    else:
        return 'UNKNOWN'


def compute_block_code(df):
    df['block_code'] = df.apply(lambda row: compute_prn_to_block_value(row.prn_code), axis=1)
    return df


def compute_daily_hour_column(df):
    df['daily_hour'] = df.apply(lambda row: round(row.ddm_timestamp_utc / (60*60)), axis=1)
    return df


def compute_time_of_day_value(time):
    if time >= 22:
        return 'N'
    elif time >= 16:
        return 'A'
    elif time >= 10:
        return 'D'
    elif time >= 4:
        return 'M'
    else:
        return 'N'
    

def compute_time_of_day(df):
    df['time_of_day'] = df.apply(lambda row: compute_time_of_day_value(row.daily_hour), axis=1)
    return df


def scale_sr_values(df):
    min_sr = df['sr'].min()
    df['sr'] = df['sr'].apply(lambda x: x - min_sr)
    return df


def filter_location(df, location, cygnss=True):
    if cygnss:
        filtered_df = df[df.sp_lat < location[0]]
        filtered_df = filtered_df[filtered_df.sp_lat > location[2]]
        filtered_df = filtered_df[filtered_df.sp_lon < location[3]]
        filtered_df = filtered_df[filtered_df.sp_lon > location[1]]
    else:
        filtered_df = df[df.lat < location[0]]
        filtered_df = filtered_df[filtered_df.lat > location[2]]
        filtered_df = filtered_df[filtered_df.long < location[3]]
        filtered_df = filtered_df[filtered_df.long > location[1]]
    
    return filtered_df


def filter_quality_flags_1(df):
    df['qf_ok'] = df.apply(
        lambda row: (2 or 4 or 5 or 8 or 16 or 17) not in generate_qf_list(int(row.quality_flags)), axis=1)
    df = df[df['qf_ok']]
    return df


def filter_quality_flags_2(df):
    res_df = df
    res_df['qf2_ok'] = res_df.apply(
        lambda row: (1 or 2) not in generate_qf_list(int(row.quality_flags_2)), axis=1)  # Remember to check which qfs
    res_df = res_df[res_df['qf2_ok']]
    return res_df

def filter_cygnss_qf(df):
    remove_mask = int('11000000010011010', 2)

    return df[df['quality_flags'] & remove_mask == 0]


def remove_fill_values(df, raw_data):
    keys = list(raw_data.keys())
    keys.remove('ddm_timestamp_utc')
    keys.remove('spacecraft_num')
    filtered_df = df

    # Remove rows containing fill values
    for k in keys:
        key = raw_data[k]
        fv = key._FillValue
        filtered_df = filtered_df[filtered_df[k] != fv]

    return filtered_df


# Returning the same df if no specific features are selected
def select_df_features(df, feature_list):
    if len(feature_list) > 0:
        return df[feature_list]
    else:
        return df


def store_df_as_csv(df, storage_path, file_name):
    storage_dir = Path(storage_path)
    storage_dir.mkdir(parents=True, exist_ok=True)
    df.to_csv(storage_dir / file_name, index=False)


In [5]:
### DATA PROCESSING MAIN FUNCTIONS ###
def raw_df_processing(df, location, qf1_removal=False, qf2_removal=False):
    res_df = df

    #print('Filtering the DataFrame based on provided location...')
    res_df = filter_location(res_df, location)

    if qf1_removal:
        print('Removing bad quality values...')
        rows_before_removal = res_df.shape[0]
        #res_df = filter_quality_flags_1(res_df)
        res_df = filter_cygnss_qf(res_df)
        rows_after_removal = res_df.shape[0]
        print('Removed ' + str(rows_before_removal - rows_after_removal) + ' rows of bad overall quality...')

    if qf2_removal:
        print('Removing more bad quality values...')
        rows_before_removal = res_df.shape[0]
        res_df = filter_quality_flags_2(res_df)
        rows_after_removal = res_df.shape[0]
        print('Removed ' + str(rows_before_removal - rows_after_removal) + ' rows of bad overall quality...')

    #print('Computing surface reflectivity values for all rows...')
    res_df = compute_surface_reflectivity(res_df)

    #print('Adding column displaying hours after January 1st 2020...')
    res_df = compute_hours_after_jan(res_df)

    #print('Computing unique track ids for all rows...')
    res_df = compute_unique_track_ids(res_df)

    return res_df


def process_monthly_df(retrieval_folder, year, month, location, qf1_removal=True, qf2_removal=False):
    monthly_df = pd.DataFrame()
    num_of_days = monthrange(year, month)[1]

    for i in range(num_of_days):
        csv_path = retrieval_folder + str(month).zfill(2) + '/raw_main_df_' + str(year) + '_' + str(month).zfill(2) + '_' + \
                   str(i + 1) + 'of' + str(num_of_days) + '.csv'
        #print('#######################################')
        #print('Collecting csv file number ' + str(i + 1) + ' of ' + str(num_of_days) + '...')
        daily_df = pd.read_csv(csv_path)
        #print('***Processing the data***')
        daily_df = raw_df_processing(daily_df, location, qf1_removal, qf2_removal)
        monthly_df = monthly_df.append(daily_df, ignore_index=True)
        #print('#######################################\n')
    return monthly_df


In [6]:
def process_cygnss_df(location, year, start_month=1, end_month=12):
    location_string = str(location[0]) + '-' + str(location[1]) + '-' + str(location[2]) + '-' + str(location[3])
    yearly_cygnss_df = pd.DataFrame()

    for i in tqdm(range(start_month, end_month+1)):
        print('*' * 39)
        print('*' * 14 + f'{month_name[i]:^11}' + '*' * 14)
        print('*' * 39 + '\n')
        monthly_df = process_monthly_df(cygnss_retrieval_folder, year, i, location, False, False)
        yearly_cygnss_df = yearly_cygnss_df.append(monthly_df, ignore_index=True)
    
    file_name = 'CYGNSS' + str(year) + '-withQFs-[' + location_string + '].csv'
    store_df_as_csv(yearly_cygnss_df, processed_cygnss_storage_folder, file_name)
    return yearly_cygnss_df


def get_cygnss_df(location, year):
    location_string = str(location[0]) + '-' + str(location[1]) + '-' + str(location[2]) + '-' + str(location[3])
    file_name = 'CYGNSS' + str(year) + '-withQFs-[' + location_string + '].csv'
    
    try:
        return pd.read_csv('/Users/madsrindal/Desktop/Intervals/' + location_string + '/' + file_name)
        #return pd.read_csv('/Volumes/MadsDrive/Master/Processed Files/' + location_string + '/' + file_name)
    except:
        return process_cygnss_df(location, year)
        #return process_cygnss_df(location, year, start_month=11, end_month=12)


In [7]:
# Set parameters
process_single_area = True
process_single_year = False

#single_area = 'WestIranFlood2020'
#single_location = [35, 49.5, 30, 54.5]


single_area = 'CentralAfrica'
single_location = [-7, 23, -12, 28]


if process_single_year:
    years = [single_year]
else:
    years = [2019, 2020, 2021]

if process_single_area:
    all_areas = [single_area]
    all_locations = [single_location]
else:
    all_areas = ['CentralAfrica', 'Brazil', 'Australia', 'IranPakistan', 'India']
    all_locations = [[-7, 23, -12, 28], [-5, -42, -10, -37], [-22, 117, -27, 122], [31, 59, 26, 64], [24, 80, 19, 85]]

locations_df = pd.DataFrame.from_dict({'area': all_areas, 'location': all_locations})

In [8]:
if load_cygnss_boolean:
    
    for desired_year in years:
        cygnss_retrieval_folder = '/Volumes/DACOTA HDD/Semester Project CSV/CYGNSS ' + str(desired_year) + '-'

        for loc in locations_df['location']:
            location_string = str(loc[0]) + '-' + str(loc[1]) + '-' + str(loc[2]) + '-' + str(loc[3])
            #processed_cygnss_storage_folder = '/Volumes/MadsDrive/Master/Processed Files/' + location_string
            processed_cygnss_storage_folder = '/Users/madsrindal/Desktop/Intervals/' + location_string
            get_cygnss_df(loc, desired_year)
    

In [ ]:
### LOAD SMAP DATAFRAME ###

In [9]:
### INTERPOLATION FUNCTIONS ###

def interpolate_ml(df: pd.DataFrame, target_value='swvl1') -> LinearNDInterpolator:
    coordinates = list(zip(list(df['time']), list(df['lat']), list(df['long'])))
    target = df[target_value]
    interpolation_function = LinearNDInterpolator(coordinates, target)
    return interpolation_function


def filter_nan_smap_sm(df):
    try:
        df['smap_sm'] = df['smap_sm'].apply(lambda x: x.item(0))
    except:
        print('SMAP_SM value was already of type: float')
    df = df.dropna()
    return df


def filter_nan_smap_vo(df):
    try:
        df['smap_vo'] = df['smap_vo'].apply(lambda x: x.item(0))
    except:
        print('SMAP_VO value was already of type: float')
    df = df.dropna()
    return df


def filter_nan_smap_sr(df):
    try:
        df['smap_surface_roughness'] = df['smap_surface_roughness'].apply(lambda x: x.item(0))
    except:
        print('SMAP_Surface_Roughness value was already of type: float')
    df = df.dropna()
    return df


def get_smap(path: str, printing=False):

    ds = nc.Dataset(path)
    sm = ds['Soil_Moisture_Retrieval_Data_AM']

    lats = np.array(sm['latitude']).flatten()
    lons = np.array(sm['longitude']).flatten()
    times = np.array(sm['tb_time_utc']).flatten()
    sms = np.array(sm['soil_moisture']).flatten()
    qfs = np.array(sm['retrieval_qual_flag']).flatten()
    vos = np.array(sm['vegetation_opacity']).flatten()
    srs = np.array(sm['roughness_coefficient']).flatten()
    
    df = pd.DataFrame()
    df['lat'] = lats
    df['long'] = lons
    df['time'] = times
    df['smap_sm'] = sms
    df['retrieval_qfs'] = qfs
    df['surface_roughness'] = srs
    df['vegetation_opacity'] = vos

    # Filter out missing values
    smap_df = df[df['smap_sm'] != -9999.0]

    if len(smap_df) > 0 and printing:
        print('Number of missing values:', len(df) - len(smap_df))
        print('Number of data points with value:', len(smap_df))
        index = list(smap_df['smap_sm']).index(max(list(smap_df['smap_sm'])))
        print("Peak SM value:", list(smap_df['smap_sm'])[index])
        print("Peak SM value at: (" + str(list(smap_df['lat'])[index]) + ", " + str(list(smap_df['long'])[index]) + ")")

    return smap_df


def conv(t):
    try:
        return pd.Timestamp(t)
    except:
        return pd.Timestamp(t.split('.')[0] + '.000Z')


def convert_time(df: pd.DataFrame) -> pd.DataFrame:
    ref_date = pd.Timestamp('2019-01-01T00:00:00.000Z')

    df['time'] = df['time'].apply(lambda t: conv(t))
    df['time'] = df['time'].apply(lambda t: (t - ref_date).days * 24 + (t - ref_date).seconds / 3600)
    return df


def get_smap_df_year(root_dir: str, year: int, convert_time_hours=True) -> pd.DataFrame:
    first = True
    all_paths = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if not first:
                all_paths.append(os.path.join(subdir, file))
            else:
                first = False

    smap_df = pd.DataFrame()

    for path in tqdm(all_paths):
        path_split = path.split('_')
        #print(path_split)
        if len(path_split) > 6:
            current_year = int(path_split[4][:4])

            if current_year == year:
                current_df = get_smap(path)
                smap_df = smap_df.append(current_df)

    if convert_time_hours:
        smap_df = convert_time(smap_df)

    return smap_df


def get_smap_main(root_path: str, years: list, months: list, days: list) -> pd.DataFrame:
    first = True
    sub_dirs = []
    filenames = []

    for dir_name, sub_dir_list, file_list in os.walk(root_path):
        if first:
            sub_dirs = sub_dir_list
            first = False
        else:
            filenames.append(file_list[0])
    
    smap_df = pd.DataFrame()
    
    for i in tqdm(range(len(sub_dirs))):
        current_day = int(filenames[i].split('_')[4][6:8])
        current_month = int(filenames[i].split('_')[4][4:6])
        current_year = int(filenames[i].split('_')[4][:4])
        
        if (current_day in days) and (current_year in years) and (current_month in months):
            current_path = root_path + '/' + sub_dirs[i] + '/' + filenames[i]
            current_df = get_smap(current_path)
            smap_df = smap_df.append(current_df)
    
    smap_df = convert_time(smap_df)
    return smap_df


def interpolate_smap_df(cygnss_df, interpolation_function, column_name):
    cygnss_df[column_name] = cygnss_df.progress_apply(lambda row: interpolation_function(row.hours_after_jan_2019, row.sp_lat, row.sp_lon), axis=1)
    return cygnss_df


def filter_smap_qfs(smap_df):
    return smap_df.loc[(smap_df['retrieval_qfs'] == 0) | (smap_df['retrieval_qfs'] == 8)]


In [10]:
if load_smap_boolean:
    
    for area in locations_df['area']:
        loc = list(locations_df['location'][locations_df['area'] == area])[0]
        location_string = str(loc[0]) + '-' + str(loc[1]) + '-' + str(loc[2]) + '-' + str(loc[3])
        
        #rootdir = '/Users/madsrindal/Desktop/Master project/SMAP/' + area + '-3years'
        rootdir = '/Users/madsrindal/Desktop/Master project/SMAP/CentralAfrica'
        
        #processed_smap_storage_folder = '/Volumes/MadsDrive/Master/Processed Files/' + location_string
        processed_smap_storage_folder = '/Users/madsrindal/Desktop/Intervals/' + location_string

        #smap_df = get_smap_df_year(rootdir, desired_year, convert_time_hours=True
        smap_df = get_smap_main(rootdir, years, range(13), range(100))

        file_name = 'SMAP-all3Years-withQFs-[' + location_string + '].csv'
        store_df_as_csv(smap_df, processed_smap_storage_folder, file_name)

100%|██████████| 1063/1063 [02:58<00:00,  5.96it/s]


In [ ]:
### INTERPOLATE SMAP DATAFRAME ###

In [11]:
def get_hours_after_jan_2019(old_hours, year):
    if year == 2019:
        return old_hours
    elif year == 2020:
        return old_hours + 24*365
    else:
        return old_hours + 24*365 + 24*366 # 366 fordi 2020 var skuddår


def fix_hours_after_jan_2019(df):
    df['hours_after_jan_2019'] = df.progress_apply(
        lambda row: get_hours_after_jan_2019(row.hours_after_jan_2020, row.year), axis=1)
    return df
    

In [12]:
if interpolate_dfs_boolean:

    for area in locations_df['area']:
        loc = list(locations_df['location'][locations_df['area'] == area])[0]
        location_string = str(loc[0]) + '-' + str(loc[1]) + '-' + str(loc[2]) + '-' + str(loc[3])
        interpolated_storage_folder = '/Users/madsrindal/Desktop/Intervals/' + location_string
        
        cygnss_df = pd.DataFrame()
        print('Reading CYGNSS files...')
        for year in years:
            cygnss_df_tmp = pd.read_csv(interpolated_storage_folder + '/' + 'CYGNSS' + str(year) + '-withQFs-[' + location_string + '].csv')
            cygnss_df_tmp['year'] = year
            cygnss_df = cygnss_df.append(cygnss_df_tmp, ignore_index=True)
        
        print('Fix hours after jan 2019...')
        cygnss_df = fix_hours_after_jan_2019(cygnss_df)
        cygnss_df.drop('hours_after_jan_2020', inplace=True, axis=1)
        
        print('Reading SMAP file...')
        #smap_df = pd.read_csv(interpolated_storage_folder + '/' + 'SMAP' + str(desired_year) + '-withQFs-[' + location_string + '].csv')
        smap_df = pd.read_csv('/Users/madsrindal/Desktop/Intervals/' + location_string + '/SMAP-all3Years-withQFs-[' + location_string + '].csv')
        smap_df = filter_smap_qfs(smap_df)

        # Creating Interpolation Functions
        print('Creating interpolation function for soil moisture...')
        start_time = time.time()
        smap_sm_func = interpolate_ml(smap_df, target_value='smap_sm')
        print('Created smap_sm interpolation function in ' + str(time.time()-start_time) + ' seconds...\n')
        
        start_time = time.time()
        print('Creating interpolation function for vegetation opacity...')
        smap_vo_func = interpolate_ml(smap_df, target_value='vegetation_opacity')
        print('Created smap_vo interpolation function in ' + str(time.time()-start_time) + ' seconds...\n')
        
        start_time = time.time()
        print('Creating interpolation function for surface roughness...')
        smap_roughness_func = interpolate_ml(smap_df, target_value='surface_roughness')
        print('Created smap_roughness interpolation function in ' + str(time.time()-start_time) + ' seconds...\n')
        

Reading CYGNSS files...
Fix hours after jan 2019...


100%|██████████| 3361360/3361360 [01:09<00:00, 48262.29it/s]


Reading SMAP file...
Creating interpolation function for soil moisture...
Created smap_sm interpolation function in 100.9637701511383 seconds...

Creating interpolation function for vegetation opacity...
Created smap_vo interpolation function in 101.3816409111023 seconds...

Creating interpolation function for surface roughness...
Created smap_roughness interpolation function in 102.86946487426758 seconds...



In [13]:
### Quick fix ###

best_loc = [-10, 27, -10.5, 27.5]
forest_loc = [-10, 23.5, -10.5, 24]
worst_loc = [-8, 27.5, -8.5, 28]

# Filter CYGNSS QFs
cygnss_df = filter_cygnss_qf(cygnss_df)

# Select smaller area
best_cell_df = filter_location(cygnss_df, best_loc)
forest_cell_df = filter_location(cygnss_df, forest_loc)
worst_cell_df = filter_location(cygnss_df, worst_loc)

In [14]:
# Interpolating Data Frame
print('Interpolating SMAP soil moisture values...')
start_time = time.time()
best_interpolated_df = interpolate_smap_df(best_cell_df, smap_sm_func, 'smap_sm')
worst_interpolated_df = interpolate_smap_df(worst_cell_df, smap_sm_func, 'smap_sm')
forest_interpolated_df = interpolate_smap_df(forest_cell_df, smap_sm_func, 'smap_sm')
print('Interpolated smap_sm data frame in ' + str(time.time()-start_time) + ' seconds...\n')

print('Interpolating SMAP vegetation opacity values...')
start_time = time.time()
best_interpolated_df = interpolate_smap_df(best_interpolated_df, smap_vo_func, 'smap_vo')
worst_interpolated_df = interpolate_smap_df(worst_interpolated_df, smap_vo_func, 'smap_vo')
forest_interpolated_df = interpolate_smap_df(forest_interpolated_df, smap_vo_func, 'smap_vo')
print('Interpolated smap_vo data frame in ' + str(time.time()-start_time) + ' seconds...\n')

print('Interpolating SMAP surface roughness values...')
start_time = time.time()
best_interpolated_df = interpolate_smap_df(best_interpolated_df, smap_roughness_func, 'smap_surface_roughness')
worst_interpolated_df = interpolate_smap_df(worst_interpolated_df, smap_roughness_func, 'smap_surface_roughness')
forest_interpolated_df = interpolate_smap_df(forest_interpolated_df, smap_roughness_func, 'smap_surface_roughness')
print('Interpolated smap_roughness data frame in ' + str(time.time()-start_time) + ' seconds...\n')

Interpolating SMAP soil moisture values...


100%|██████████| 30174/30174 [02:03<00:00, 243.36it/s] 


Interpolated smap_sm data frame in 281.4665811061859 seconds...

Interpolating SMAP vegetation opacity values...


100%|██████████| 30174/30174 [01:59<00:00, 253.42it/s] 


Interpolated smap_vo data frame in 275.9295828342438 seconds...

Interpolating SMAP surface roughness values...


100%|██████████| 30174/30174 [01:57<00:00, 256.08it/s] 

Interpolated smap_roughness data frame in 275.00737500190735 seconds...



In [15]:
## Save quick fix tables ##
file_name_best = 'BestInterpolatedDF-Congo-' + str(best_loc) + '.csv'
file_name_worst = 'WorstInterpolatedDF-Congo-' + str(worst_loc) + '.csv'
file_name_forest = 'ForestInterpolatedDF-Congo' + str(forest_loc) + '.csv'

store_df_as_csv(best_interpolated_df, interpolated_storage_folder, file_name_best)
store_df_as_csv(worst_interpolated_df, interpolated_storage_folder, file_name_worst)
store_df_as_csv(forest_interpolated_df, interpolated_storage_folder, file_name_forest)

In [ ]:
file_name = 'InterpolatedDF-withCYGNSSQFs-[' + location_string + '].csv'
store_df_as_csv(interpolated_df, interpolated_storage_folder, file_name)

In [ ]:
test_df = interpolated_df.copy()

In [ ]:
before = test_df.shape[0]
filtered_df = filter_nan_smap_sm(test_df)
filtered_df = filter_nan_smap_vo(filtered_df)
filtered_df = filter_nan_smap_sr(filtered_df)
after = filtered_df.shape[0]

print('Removed ' + str(before-after) + ' rows containing nan values')

In [ ]:
### ANNET ###

In [ ]:
print('8: ', str(bin(8)).split('b')[1].zfill(16))
print('9: ', str(bin(9)).split('b')[1].zfill(16))
print('13: ', str(bin(13)).split('b')[1].zfill(16))

print('2: ', str(bin(2)).split('b')[1].zfill(16))

print(0 % 2)

print(type(bin(8)[-1]))

In [ ]:
### TIME SERIES - FLOOD DETECTION

In [ ]:
import pandas as pd
from time import time
import numpy as np
from tqdm import tqdm
import os
import netCDF4 as nc
import datetime
from matplotlib import pyplot as plt, rcParams
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib
from scipy.ndimage import gaussian_filter
import xarray as xr
import random
from scipy.interpolate import LinearNDInterpolator
from itertools import product
from scipy.signal import savgol_filter
tqdm.pandas()

from matplotlib import pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import numpy as np
from matplotlib.patches import Rectangle
from random import random


# Main time series for loop
def time_series_analysis(cygnss_df, smap_df, area, moving_window=2, interval=10, use_median=True, 
                         plot=True, area_name=None, save=None, use_smoothening=False, sigma=None,
                         error_bar_scale=10):
    
    cygnss_df = filter_cygnss_df(cygnss_df, area)
    smap_df = filter_cygnss_df(smap_df, area)
    
    first_day = cygnss_df['day_of_year'].min()
    last_day = cygnss_df['day_of_year'].max()
    
    surface_ref = {}
    surface_ref_std = {}
    soil_moisture = {}
    soil_moisture_std = {}
    
    for day in np.arange(first_day, last_day - moving_window, moving_window):
        # Filter CYGNSS
        current_cygnss = filter_cygnss_day(cygnss_df, day, day + interval)
        if use_smoothening:
            current_cygnss = grid_box(current_cygnss, 'sr', True)
            current_cygnss = smoothening(current_cygnss, area, sigma, 'sr')
        
        # Filter SMAP
        current_smap = filter_smap_day(smap_df, day*24, (day + interval)*24)
        
        if use_median:
            current_sr = current_cygnss['sr'].median()
            current_sr_std = current_cygnss['sr'].mad()
        else:
            current_sr = current_cygnss['sr'].mean()
            current_sr_std = current_cygnss['sr'].std()
        
        if len(current_smap) > 0:
            current_sm = current_smap['smap_sm'].mean()
            current_sm_std = current_smap['smap_sm'].std()
        else:
            # print('EMPTY SMAP from day:', day, 'to day:', day + interval)
            current_sm = np.mean(list(soil_moisture.values()))
            current_sm_std = 0
        
        surface_ref[day] = current_sr
        surface_ref_std[day] = current_sr_std
        soil_moisture[day] = current_sm
        soil_moisture_std[day] = current_sm_std
    
    if plot:
        
        error_bar_top = []
        error_bar_bottom = []
        
        for i in range(len(list(surface_ref.values()))):
            error_bar_top.append(list(surface_ref.values())[i] + list(surface_ref_std.values())[i]/error_bar_scale)
            error_bar_bottom.append(list(surface_ref.values())[i] - list(surface_ref_std.values())[i]/error_bar_scale)
        
        fig = plt.figure(figsize=(20, 6))
        ax = plt.axes()
        ax.grid()
        
        ax.plot(list(surface_ref.keys()), list(surface_ref.values()), color="red", marker="o")
        ax.fill_between(list(surface_ref_std.keys()), error_bar_top, error_bar_bottom, alpha=.2, linewidth=0, color='red')
        label_size = 20
        ax.set_xlabel("Day after 1st of Jan 2019", fontsize=label_size)
        ax.set_ylabel("SR [dB]", color="red", fontsize=label_size)
        

        ax2 = ax.twinx()
        ax2.grid(False)
        ax2.plot(list(soil_moisture.keys()), list(soil_moisture.values()), color="blue", marker="o")
        ax2.set_ylabel("SM [cm^3/cm^3]", color="blue", fontsize=label_size)
        ax.legend(['Surface Reflectivity (SR)', 'std SR/' + str(error_bar_scale)], loc=2)
        ax2.legend(['Soil Moisture (SM)'], loc=3)
        
        if area_name is not None:
            plt.title(area_name + " Correlation: " + str(round(pd.Series(surface_ref.values()).corr(pd.Series(soil_moisture.values())), 3)), fontsize=22)
            if save is not None:
                plt.savefig('/Users/vegardhaneberg/Desktop/Plots Master/Time Series/' + str(area_name) + '.png', format='png')
        else:
            plt.title("Correlation: " + str(round(pd.Series(surface_ref.values()).corr(pd.Series(soil_moisture.values())), 3)), fontsize=18)
            if save is not None:
                random_num = lambda: random.randint(0, 255)
                random_name = '%02X%02X%02X' % (random_num(), random_num(), random_num())
                plt.savefig('/Users/vegardhaneberg/Desktop/Plots Master/Time Series/Without Name' + random_name + '.png', format='png')
        
        plt.show()
        
    return surface_ref, surface_ref_std, soil_moisture, soil_moisture_std


def filter_cygnss_df(df: pd.DataFrame, area: dict) -> pd.DataFrame:
    if 'sp_lat' in df.columns:
        new_df = df[df['sp_lat'] <= area['north']]
        new_df = new_df[new_df['sp_lat'] >= area['south']]
        new_df = new_df[new_df['sp_lon'] >= area['west']]
        new_df = new_df[new_df['sp_lon'] <= area['east']]
    else:
        new_df = df[df['lat'] <= area['north']]
        new_df = new_df[new_df['lat'] >= area['south']]
        new_df = new_df[new_df['long'] >= area['west']]
        new_df = new_df[new_df['long'] <= area['east']]

    return new_df


def filter_cygnss_day(df, start_day, end_day):
    filtered_df = df[df['day_of_year'] >= start_day]
    filtered_df = filtered_df[filtered_df['day_of_year'] < end_day]
    
    return filtered_df


def filter_smap_day(df, start_hour, end_hour):
    filtered_df = df[df['time'] >= start_hour]
    filtered_df = filtered_df[filtered_df['time'] < end_hour]
    return filtered_df


In [ ]:
area_dict = {'north': 35, 'west': 49.5, 'south': 30, 'east': 54.5}
step_size = 2
num_days_avg = 2
use_median = False
plot_time_series = True
plot_name = 'Plot Name'

In [ ]:
cyg_df_ts = pd.read_csv('/Users/madsrindal/Desktop/Intervals/35-49.5-30-54.5/CYGNSS2020-withQFs-[35-49.5-30-54.5].csv').rename(columns={'sp_lat': 'lat', 'sp_lon':'long'})
smap_df_ts = pd.read_csv('/Users/madsrindal/Desktop/Intervals/35-49.5-30-54.5/SMAP2020-withQFs-[35-49.5-30-54.5].csv')


In [ ]:
cyg_df_ts.head()

In [ ]:
time_series_analysis(cyg_df_ts, smap_df_ts, area_dict, step_size, num_days_avg, use_median, plot_time_series, plot_name)


In [ ]:
a = {339: 1, 340: 2}
print(list(a.keys()))

In [ ]:
def round_nearest(x, a):
    return round(x / a) * a


def grid_box(df, resolution, target_value='sr', use_median=False):
    #df['lat'] = df['lat'].apply(lambda x: round(round_nearest(x, resolution), 1))
    df['lat'] = df['lat'].apply(lambda x: round_nearest(x, resolution), 1)
    #df['long'] = df['long'].apply(lambda x: round(round_nearest(x, resolution), 1))
    df['long'] = df['long'].apply(lambda x: round_nearest(x, resolution), 1)

    if use_median:
        df = df.groupby(['long', 'lat'], as_index=False)[target_value].median()
    else:
        df = df.groupby(['long', 'lat'], as_index=False)[target_value].mean()

    return df

In [ ]:
def get_smap(path: str, printing=False):

    ds = nc.Dataset(path)
    sm = ds['Soil_Moisture_Retrieval_Data_AM']
    
    print(sm['latitude'])
    
    lats = np.array(sm['latitude']).flatten()
    lons = np.array(sm['longitude']).flatten()
    times = np.array(sm['tb_time_utc']).flatten()
    sms = np.array(sm['soil_moisture']).flatten()
    qfs = np.array(sm['retrieval_qual_flag']).flatten()
    
    df = pd.DataFrame()
    df['lat'] = lats
    df['long'] = lons
    df['time'] = times
    df['smap_sm'] = sms
    df['retrieval_qfs'] = qfs

    return df


In [ ]:
path = '/Users/madsrindal/Downloads/SMAP_L3_SM_P_20190102_R18290_001_HEGOUT.nc'

In [ ]:
start = time.time()
df = get_smap(path)
print(time.time()-start)

In [ ]:
def test_code(path: str):

    ds = nc.Dataset(path)
    sm = ds['Soil_Moisture_Retrieval_Data_AM']

    latitudes = []
    longitudes = []
    moistures = []
    time_values = []
    qfs = []

    for lat in tqdm(range(len(sm['latitude']))):
        for long in range(len(sm['longitude'][lat])):
            latitudes.append(sm['latitude'][lat][long])
            longitudes.append(sm['longitude'][lat][long])
            moistures.append(sm['soil_moisture'][lat][long])
            time_values.append(sm['tb_time_utc'][lat][long])
            qfs.append(sm['retrieval_qual_flag'][lat][long])

    return latitudes, longitudes, moistures, time_values, qfs


def filter_smap_qfs(smap_df):
    return smap_df.loc[(smap_df['retrieval_qfs'] == 0) | (smap_df['retrieval_qfs'] == 8)]

In [ ]:
latitudes, longitudes, moistures, times, qfs = test_code(path)

In [ ]:
print(len(latitudes))
print(len(longitudes))
print(len(moistures))

In [ ]:
pth = '/Users/madsrindal/Downloads/SMAP_L3_SM_P_20190102_R18290_001_HEGOUT.nc'
ds = nc.Dataset(pth)
sm = ds['Soil_Moisture_Retrieval_Data_AM']

In [ ]:
print(type(sm['soil_moisture'][2][0]))

lats = np.array(sm['latitude']).flatten()
lons = np.array(sm['longitude']).flatten()
sms = np.array(sm['soil_moisture']).flatten()
qs = np.array(sm['retrieval_qual_flag']).flatten()

print(type(list(sms)[0]))

In [ ]:
def conv_fill_value(value):
    try:
        return np.float32(value)
    except:
        return -9999.0

In [ ]:
res = conv_fill_value(12)
print(type(res))

In [ ]:
test_df = pd.DataFrame()
test_df['lat'] = latitudes
test_df['long'] = longitudes
test_df['smap_sm'] = moistures

#test_df['smap_sm'] = test_df['smap_sm'].progress_apply(lambda x: conv_fill_value(x))

In [ ]:
df = df.drop(columns=['time', 'retrieval_qfs'])

In [ ]:
df.shape

In [ ]:
test_df.shape

In [ ]:
mads_df = df.copy()
vegard_df = test_df.copy()

In [ ]:
mads_df.head()

In [ ]:
vegard_df.head()

In [ ]:
mads_df = mads_df[(mads_df['lat'] != -9999.0) & (mads_df['long'] != -9999.0)]
vegard_df = vegard_df[(vegard_df['lat'] != -9999.0) & (vegard_df['long'] != -9999.0)]

In [ ]:
mads_df.shape

In [ ]:
mads_df['smap_sm'].value_counts()

In [ ]:
vegard_df['smap_sm'].value_counts()

In [ ]:
vegard_df['smap_sm'] = vegard_df['smap_sm'].apply(lambda x: str(x))
vegard_df['smap_sm'] = vegard_df['smap_sm'].apply(lambda x: -9999.0 if x=='--' else x)

In [ ]:
vegard_df.head()

In [ ]:
vegard_df['smap_sm'] = vegard_df['smap_sm'].apply(lambda x: float(x))

In [ ]:
vegard_df = vegard_df.dropna()

In [ ]:
vegard_df.shape

In [ ]:
mads_df = mads_df[mads_df['smap_sm'] != -9999.0]
print(mads_df.shape)

In [ ]:
mads_df_list = mads_df.index.tolist()
vegard_df_list = vegard_df.index.tolist()

print('Mads_df_list: ', len(mads_df_list))
print('Vegard_df_list: ', len(vegard_df_list))

In [ ]:
main_list = list(set(mads_df_list) - set(vegard_df_list))

In [ ]:
print(len(main_list))

In [ ]:
for num in main_list:
    print(moistures[num])
    print(sms[num])
    print('x'*50)
    print('\n')

In [ ]:
df.loc[8269]['smap_sm']

In [ ]:
test_df.loc[8269]

In [ ]:
test_df.isna().sum()

In [ ]:
def get_plot_ticks(lat_values, long_values):
    min_lat = min(lat_values)
    max_lat = max(lat_values)
    min_long = min(long_values)
    max_long = max(long_values)
    
    lat_step_size = (max_lat - min_lat) / 3
    long_step_size = (max_long - min_long) / 3
    
    long_list = [min_long, min_long + long_step_size, min_long + 2 * long_step_size, max_long]
    lat_list = [min_lat, min_lat + lat_step_size, min_lat + 2 * lat_step_size, max_lat]
    
    # Rounding to two decimals
    long_list = [round(num, 2) for num in long_list]
    lat_list = [round(num, 2) for num in lat_list]
    
    return lat_list, long_list

In [ ]:
def universal_plot(df, target_value='swvl1', title=None, bar_title=None, vmin=None, vmax=None, save=None, dot_size=0.5, std=False, fig_size=None, regions=None, region_colors=None, region_names=None):
    
    if fig_size is not None:
        plt.rcParams["figure.figsize"] = fig_size
    else:
        plt.rcParams["figure.figsize"] = (8,8)
    
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    lat_list, long_list = get_plot_ticks(df['lat'], df['long'])
    ax.set_xticks(long_list, crs=ccrs.PlateCarree())
    ax.set_yticks(lat_list, crs=ccrs.PlateCarree())
    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter())
    
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    
    if std:
        cmap = 'Greys'
    else:
        cmap = 'Spectral'
        
    if vmin is not None:
        plt.scatter(df['long'], df['lat'], c=list(df[target_value]), s=dot_size, cmap=cmap, vmin=vmin, vmax=vmax)
    else:
        plt.scatter(df['long'], df['lat'], c=list(df[target_value]), s=dot_size, cmap=cmap)
    
    bar = plt.colorbar(shrink=0.7)
    bar.ax.tick_params(labelsize=15)
    if bar_title is not None:
        bar.ax.set_title(bar_title, fontsize=18)
    
    if title is not None:
        plt.title(title, fontsize=20, fontweight='book')
    
    if regions is not None:
        legend_elements = []
        
        for i in range(len(regions)):
            ax.add_patch(regions[i])
            legend_elements.append(Line2D([0], [0], color=region_colors[i], lw=3, label=region_names[i]))

        ax.legend(handles=legend_elements, loc='upper right', fontsize=16)
        
    plt.xlabel('Longitude', fontsize=18)
    plt.ylabel('Latitude', fontsize=18)
    
    if save is not None:
        plt.savefig(save, bbox_inches='tight')

    plt.show()
    

In [ ]:
import pandas as pd
from time import time
import numpy as np
from tqdm import tqdm
import os
import netCDF4 as nc
from progressbar import progressbar
import datetime
from matplotlib import pyplot as plt, rcParams
from matplotlib import pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib
from scipy.ndimage import gaussian_filter
import xarray as xr
import random
from scipy.interpolate import LinearNDInterpolator
from itertools import product
from scipy.signal import savgol_filter
tqdm.pandas()

from matplotlib import pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import numpy as np
from matplotlib.patches import Rectangle
from random import random

#%matplotlib inline

In [ ]:
universal_plot(df, 'smap_sm', fig_size=(14,8))

In [ ]:
max_value = df['smap_sm'].max()
min_value = df['smap_sm'].min()
mean_value = df['smap_sm'].mean()

print('Max: ', max_value)
print('Min: ', min_value)
print('Snitt: ', mean_value)

In [ ]:
max_value = test_df['smap_sm'].max()
min_value = test_df['smap_sm'].min()
mean_value = test_df['smap_sm'].mean()

print('Max: ', max_value)
print('Min: ', min_value)
print('Snitt: ', mean_value)

In [ ]:
df.head()